# 2021/12/12/compaired_by_kernelsize

In [ ]:
import numpy as np
import cv2 
from matplotlib import pyplot as plt
import csv
from pathlib import Path
from process_one import *

In [ ]:
kernel_size_list = [100,150,200,250,300,350,400,450]
input_dir_path = Path("data/origin_data/anno_origin/hogehogehoge")
data_path_list = list(input_dir_path.glob("*.jpg"))
annotate_dir_path = Path("data/ikeuchi_anno_data")

In [ ]:
images_dic = {}
with open("data/origin_data/anno_origin/new_roi.csv") as csvfile:
    roi = csv.reader(csvfile, delimiter=' ', quotechar='|')
    roi = list(roi)
    for row in roi[1:]:
        image, hmin, hmax, anno_data_path = row[0].split(',')
        images_dic[image] = int(hmin),int(hmax), anno_data_path

In [ ]:
def create_mask(image_path, hmin, hmax):  
    leaf_image_bgr = read_image(image_path)
    leaf_image_rgb = cv2.cvtColor(leaf_image_bgr, cv2.COLOR_BGR2RGB)
    leaf_image_bgr_crop = crop(leaf_image_bgr, hmin, hmax)
    leaf_image_rgb_crop = crop(leaf_image_rgb, hmin, hmax)
    
    img_lch = rgb2lch(leaf_image_rgb_crop)
    img_lsh = calculate_perception(img_lch)
    lsh_mask = extract_blank_area(img_lsh) & np.bitwise_not(extract_green_area(leaf_image_bgr_crop))
    return leaf_image_rgb, lsh_mask
    
def create_images_by_kernelsize(leaf_image_rgb, lsh_mask, kernel_size, hmin, hmax):
    # Visualization (heatmap)
    density_img = normalize(cv2.blur(255 - lsh_mask, (kernel_size, kernel_size)), v_min=DENSITY_MIN)
    heatmap_img = cv2.applyColorMap(density_img, cv2.COLORMAP_JET)
    overlay_heatmap = alpha_blend(leaf_image_rgb, heatmap_img, hmin, hmax)
    overlay_heatmap = cv2.cvtColor(overlay_heatmap, cv2.COLOR_BGR2RGB)

    # Visualization (contour)
    contour_img = cv2.applyColorMap(discretize(density_img), cv2.COLORMAP_JET)
    overlay_contour = alpha_blend(leaf_image_rgb, contour_img, hmin, hmax)
    overlay_contour = cv2.cvtColor(overlay_contour,cv2.COLOR_BGR2RGB)
    
    return overlay_contour, overlay_heatmap

def visualize_images(path):
    hmin, hmax, anno_data_path = images_dic[path.name]
    # call_annoimage
    anno_image = cv2.imread(anno_data_path)
    anno_image = cv2.cvtColor(anno_image, cv2.COLOR_BGR2RGB)
    
    leaf_image_rgb, lsh_mask = create_mask(str(path), hmin, hmax)
    
    # binalization_image
    lsh_mask_bgr = gray2bgr(lsh_mask, leaf_image_rgb.shape, hmin, hmax)
    
    # crate_images_by_kernelsize
    images_bykernel_list = []
    for kernel_size in kernel_size_list:
        overlay_contour, overlay_heatmap = create_images_by_kernelsize(leaf_image_rgb, lsh_mask, kernel_size, hmin, hmax)
        images_bykernel_list.append((overlay_contour, overlay_heatmap))
    # visualization
    fig, axes = plt.subplots(2, len(kernel_size_list)+3,figsize=(50,20))
    fig.suptitle(f"image_name : {path.name}", fontsize=20)
    # 0,1,2番目にそれぞれ、元画像、
    axes[0][0].imshow(leaf_image_rgb)
    axes[1][0].imshow(leaf_image_rgb)
    axes[0][1].imshow(lsh_mask_bgr)
    axes[1][1].imshow(lsh_mask_bgr)
    axes[0][0].imshow(leaf_image_rgb)
    axes[1][0].imshow(leaf_image_rgb)
    axes[1][0].imshow(leaf_image_rgb)
    axes[1][0].imshow(leaf_image_rgb)
    axes[1][0].imshow(leaf_image_rgb)
    axes[1][2].imshow(anno_image)
    for i in range(len(kernel_size_list)):
        axes[0][i+3].set_title(f"k_size = {kernel_size_list[i]}",fontsize = 20)
        axes[0][i+3].imshow(images_bykernel_list[i][0])
        axes[1][i+3].imshow(images_bykernel_list[i][1])
    plt.show()

## Visualize images

In [ ]:
for path in data_path_list:
    visualize_images(path)